# Rolling Horizon

Solve large operational problems by decomposing the time horizon into sequential segments.

This notebook introduces:

- **Rolling horizon optimization**: Divide time into overlapping segments
- **State transfer**: Pass storage states and flow history between segments
- **When to use**: Memory limits, operational planning with limited foresight

We use a realistic district heating system with CHP, boiler, and storage to demonstrate the approach.

## Setup

In [1]:
import timeit

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import flixopt as fx

fx.CONFIG.notebook()

flixopt.config.CONFIG

## Load Time Series Data

We use real-world district heating data at 15-minute resolution (two weeks):

In [2]:
import pathlib

# Load time series data (15-min resolution)
notebook_dir = pathlib.Path('data')
if not notebook_dir.exists():
    notebook_dir = (
        pathlib.Path(__file__).parent / 'data' if '__file__' in dir() else pathlib.Path('docs/notebooks/data')
    )

data = pd.read_csv(notebook_dir / 'Zeitreihen2020.csv', index_col=0, parse_dates=True).sort_index()
data = data['2020-01-01':'2020-01-14 23:45:00']  # Two weeks

timesteps = data.index

# Extract profiles
electricity_demand = data['P_Netz/MW'].to_numpy()
heat_demand = data['Q_Netz/MW'].to_numpy()
electricity_price = data['Strompr.€/MWh'].to_numpy()
gas_price = data['Gaspr.€/MWh'].to_numpy()

print(f'Timesteps: {len(timesteps)} ({len(timesteps) / 96:.0f} days at 15-min resolution)')
print(f'Heat demand: {heat_demand.min():.1f} - {heat_demand.max():.1f} MW')
print(f'Electricity price: {electricity_price.min():.1f} - {electricity_price.max():.1f} €/MWh')

Timesteps: 1344 (14 days at 15-min resolution)
Heat demand: 122.2 - 254.3 MW
Electricity price: -3.3 - 72.6 €/MWh


In [3]:
def build_system(timesteps, heat_demand, electricity_demand, electricity_price, gas_price):
    """Build a district heating system with CHP, boiler, and storage."""
    fs = fx.FlowSystem(timesteps)

    # Effects

    # Buses
    fs.add_elements(
        fx.Bus('Electricity'),
        fx.Bus('Heat'),
        fx.Bus('Gas'),
        fx.Bus('Coal'),
        fx.Effect('costs', '€', 'Total Costs', is_standard=True, is_objective=True),
        fx.Effect('CO2', 'kg', 'CO2 Emissions'),
        fx.linear_converters.CHP(
            'CHP',
            thermal_efficiency=0.58,
            electrical_efficiency=0.22,
            status_parameters=fx.StatusParameters(effects_per_startup=24000),
            electrical_flow=fx.Flow('P_el', bus='Electricity', size=200),
            thermal_flow=fx.Flow('Q_th', bus='Heat', size=200),
            fuel_flow=fx.Flow('Q_fu', bus='Coal', size=288, relative_minimum=87 / 288, previous_flow_rate=100),
        ),
        fx.linear_converters.Boiler(
            'Boiler',
            thermal_efficiency=0.85,
            thermal_flow=fx.Flow('Q_th', bus='Heat'),
            fuel_flow=fx.Flow(
                'Q_fu',
                bus='Gas',
                size=95,
                relative_minimum=12 / 95,
                previous_flow_rate=20,
                status_parameters=fx.StatusParameters(effects_per_startup=1000),
            ),
        ),
        fx.Storage(
            'Storage',
            capacity_in_flow_hours=684,
            initial_charge_state=137,
            minimal_final_charge_state=137,
            maximal_final_charge_state=158,
            eta_charge=1,
            eta_discharge=1,
            relative_loss_per_hour=0.001,
            prevent_simultaneous_charge_and_discharge=True,
            charging=fx.Flow('Charge', size=137, bus='Heat'),
            discharging=fx.Flow('Discharge', size=158, bus='Heat'),
        ),
        fx.Source(
            'GasGrid',
            outputs=[fx.Flow('Q_Gas', bus='Gas', size=1000, effects_per_flow_hour={'costs': gas_price, 'CO2': 0.3})],
        ),
        fx.Source(
            'CoalSupply',
            outputs=[fx.Flow('Q_Coal', bus='Coal', size=1000, effects_per_flow_hour={'costs': 4.6, 'CO2': 0.3})],
        ),
        fx.Source(
            'GridBuy',
            outputs=[
                fx.Flow(
                    'P_el',
                    bus='Electricity',
                    size=1000,
                    effects_per_flow_hour={'costs': electricity_price + 0.5, 'CO2': 0.3},
                )
            ],
        ),
        fx.Sink(
            'GridSell',
            inputs=[fx.Flow('P_el', bus='Electricity', size=1000, effects_per_flow_hour=-(electricity_price - 0.5))],
        ),
        fx.Sink('HeatDemand', inputs=[fx.Flow('Q_th', bus='Heat', size=1, fixed_relative_profile=heat_demand)]),
        fx.Sink(
            'ElecDemand', inputs=[fx.Flow('P_el', bus='Electricity', size=1, fixed_relative_profile=electricity_demand)]
        ),
    )

    return fs


flow_system = build_system(timesteps, heat_demand, electricity_demand, electricity_price, gas_price)
print(f'System: {len(timesteps)} timesteps')

System: 1344 timesteps


## Full Optimization (Baseline)

First, solve the full problem as a baseline:

In [ ]:
solver = fx.solvers.HighsSolver()

start = timeit.default_timer()
fs_full = flow_system.copy()
fs_full.optimize(solver)
time_full = timeit.default_timer() - start

print(f'Full optimization: {time_full:.2f} seconds')
print(f'Cost: {fs_full.solution["costs"].item():,.0f} €')

2025-12-13 19:42:53.853 WARNING  │ FlowSystem is not connected_and_transformed. Connecting and transforming data now.


## Rolling Horizon Optimization

The `optimize.rolling_horizon()` method divides the time horizon into segments that are solved sequentially:

```
Full horizon:  |---------- 1344 timesteps (14 days) ----------|
                
Segment 1:     |==== 192 (2 days) ====|-- overlap --|
Segment 2:                |==== 192 (2 days) ====|-- overlap --|
Segment 3:                              |==== 192 (2 days) ====|-- overlap --|
...                                                  
```

Key parameters:
- **horizon**: Timesteps per segment (excluding overlap)
- **overlap**: Additional lookahead timesteps (improves storage optimization)
- **nr_of_previous_values**: Flow history transferred between segments

In [ ]:
start = timeit.default_timer()
fs_rolling = flow_system.copy()
segments = fs_rolling.optimize.rolling_horizon(
    solver,
    horizon=192,  # 2-day segments (192 timesteps at 15-min resolution)
    overlap=48,  # 12-hour lookahead
)
time_rolling = timeit.default_timer() - start

print(f'Rolling horizon: {time_rolling:.2f} seconds ({len(segments)} segments)')
print(f'Cost: {fs_rolling.solution["costs"].item():,.0f} €')

## Compare Results

In [ ]:
cost_full = fs_full.solution['costs'].item()
cost_rolling = fs_rolling.solution['costs'].item()
cost_gap = (cost_rolling - cost_full) / cost_full * 100

results = pd.DataFrame(
    {
        'Method': ['Full optimization', 'Rolling horizon'],
        'Time [s]': [time_full, time_rolling],
        'Cost [€]': [cost_full, cost_rolling],
        'Cost Gap [%]': [0.0, cost_gap],
    }
).set_index('Method')

results.style.format({'Time [s]': '{:.2f}', 'Cost [€]': '{:,.0f}', 'Cost Gap [%]': '{:.2f}'})

## Visualize: Heat Balance Comparison

In [ ]:
fs_full.statistics.plot.balance('Heat').figure.update_layout(title='Heat Balance (Full)')

In [ ]:
fs_rolling.statistics.plot.balance('Heat').figure.update_layout(title='Heat Balance (Rolling)')

## Storage State Continuity

Rolling horizon transfers storage charge states between segments to ensure continuity:

In [ ]:
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=['Full Optimization', 'Rolling Horizon']
)

# Full optimization
charge_full = fs_full.solution['Storage|charge_state'].values[:-1]  # Drop final value
fig.add_trace(go.Scatter(x=timesteps, y=charge_full, name='Full', line=dict(color='blue')), row=1, col=1)

# Rolling horizon
charge_rolling = fs_rolling.solution['Storage|charge_state'].values[:-1]
fig.add_trace(go.Scatter(x=timesteps, y=charge_rolling, name='Rolling', line=dict(color='orange')), row=2, col=1)

fig.update_yaxes(title_text='Charge State [MWh]', row=1, col=1)
fig.update_yaxes(title_text='Charge State [MWh]', row=2, col=1)
fig.update_layout(height=400, showlegend=False)
fig.show()

## Inspect Individual Segments

The method returns the individual segment FlowSystems, which can be inspected:

In [ ]:
print(f'Number of segments: {len(segments)}')
print()
for i, seg in enumerate(segments):
    start_time = seg.timesteps[0]
    end_time = seg.timesteps[-1]
    cost = seg.solution['costs'].item()
    print(
        f'Segment {i + 1}: {start_time.strftime("%Y-%m-%d %H:%M")} → {end_time.strftime("%Y-%m-%d %H:%M")} | Cost: {cost:,.0f} €'
    )

## Effect of Overlap

The overlap parameter provides lookahead for storage optimization. Let's compare different overlap values:

In [ ]:
overlaps = [0, 24, 48, 96]  # 0, 6h, 12h, 24h lookahead
overlap_results = []

for overlap in overlaps:
    fs = flow_system.copy()
    start = timeit.default_timer()
    fs.optimize.rolling_horizon(solver, horizon=192, overlap=overlap)
    elapsed = timeit.default_timer() - start
    cost = fs.solution['costs'].item()
    gap = (cost - cost_full) / cost_full * 100
    overlap_results.append(
        {'Overlap': f'{overlap} ({overlap * 15 / 60:.0f}h)', 'Time [s]': elapsed, 'Cost [€]': cost, 'Gap [%]': gap}
    )

pd.DataFrame(overlap_results).style.format({'Time [s]': '{:.2f}', 'Cost [€]': '{:,.0f}', 'Gap [%]': '{:.2f}'})

## When to Use Rolling Horizon

| Use Case | Recommendation |
|----------|----------------|
| **Memory limits** | Large problems that exceed available memory |
| **Operational planning** | When limited foresight is realistic |
| **Quick approximate solutions** | Faster than full optimization |
| **Investment decisions** | Use full optimization instead |

### Limitations

- **No investments**: `InvestParameters` are not supported (raises error)
- **Suboptimal storage**: Limited foresight may miss long-term storage opportunities
- **Global constraints**: `flow_hours_max` etc. cannot be enforced globally

## API Reference

```python
segments = flow_system.optimize.rolling_horizon(
    solver,              # Solver instance
    horizon=192,         # Timesteps per segment (e.g., 2 days at 15-min resolution)
    overlap=48,          # Additional lookahead timesteps (e.g., 12 hours)
    nr_of_previous_values=1,  # Flow history for uptime/downtime tracking
)

# Combined solution on original FlowSystem
flow_system.solution['costs'].item()

# Individual segment solutions
for seg in segments:
    print(seg.solution['costs'].item())
```

## Summary

You learned how to:

- Use **`optimize.rolling_horizon()`** to decompose large problems
- Choose **horizon** and **overlap** parameters
- Understand the **trade-offs** vs. full optimization

### Key Takeaways

1. **Rolling horizon** is useful for memory-limited or operational planning problems
2. **Overlap** improves solution quality at the cost of computation time
3. **Storage states** are automatically transferred between segments
4. Use **full optimization** for investment decisions